# Cleaing Data

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?code_challenge=oXGnc2o6ycdC7ce08rXDppK4MlftpOzEheCUF2EwhQ4&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


Enter verification code: 4/ugFjk5ZtaR5RGzPO8_UALWn5BQzUaamKJL_fDrBYa_2z4iOLhnA8fmo
If you need to use ADC, see:
  gcloud auth application-default --help

You are now logged in as [galli.giuly@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [0]:
import re
import nltk
import pandas as pd
nltk.download('stopwords')

from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Pre cleaning with SQL on BigQuery


Pre cleaning filters with SQL

The df has been pre "cleaned":

    no data with body "deleted"
    no data with body "removed"
    no data with body "Removed by reddit in response to a copyright notice."
    no NAN on body
    no empty data on body



In [0]:
# cleaning text frunction for keras model

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
SYMBOLS = re.compile(r"[^a-zA-Z0-9' ]")
STOPWORDS = set(stopwords.words('english'))

def clean_text_keras(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

def clean_text_df(df, column):
  df[column] = df[column].map(clean_text_keras)

In [0]:
# importing my final ds

!gsutil cp gs://reddit_final_results/comments_posts.pkl .

Copying gs://reddit_final_results/comments_posts.pkl...
- [1 files][374.5 MiB/374.5 MiB]                                                
Operation completed over 1 objects/374.5 MiB.                                    


In [0]:
reddit_df = pd.read_pickle('comments_posts.pkl')

In [0]:
reddit_df.head()

,subreddit,body
0,gaming,honestly share height rest shorties
1,gaming,im sure whats funnier way interpreted said act...
2,gaming,wait quelaags furysword scaled soft humanity t...
3,gaming,program use make display maps something simila...
4,gaming,ccleaner turned shit norton makes tool uninsta...


In [0]:
# cleaning my df

reddit_df['body'] = reddit_df['body'].apply(clean_text_keras)

In [0]:
reddit_df.tail()

,subreddit,body
1546813,aww,dog attempts get bed back cat claimed removed ...
1546814,aww,r aww official discord join join https discord...
1546815,aww,knows hes goodest boy removed reddit response ...
1546816,aww,collection different animal posts today mammal...
1546817,aww,bestof r aww 2017 awards thread nominations th...


In [0]:
reddit_df.to_pickle('reddit_clear_df.pkl')

In [0]:
!gsutil cp /content/reddit_clear_df.pkl gs://reddit_final_results/

Copying file:///content/reddit_clear_df.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

-
Operation completed over 1 objects/253.2 MiB.                                    
